Run this notebook to calculate the data quality metrics reported in the paper. 

In [1]:
from IPython.display import clear_output
import numpy as np
import pandas as pd
from eyemovement_data.participant import Participant
from eyemovement_data.utils import get_participant_ids

## Data Quality Metrics
We calculate five metrics to assess data quality for each participant. 

- **Average validation error:** Reported by the EyeLink. Indicates how accurately the pupil position was tracked. Should ideally be below 0.5 degrees for adults with normal vision.

- **Maximum validation error:** Reported by the EyeLink. Indicates how accurately the pupil position was tracked. Should ideally be below 1.0 degree for adults with normal vision.

- **Missing data:** Percent of gaze samples with NA values. Indicates how prevalent blinks and tracking problems are in the data. Should be as low as possible.

- **Fixation precision:** Root Mean Squared Error (RMSE) of gaze samples relative to the centroid of each classified fixation group. Reflects the spatial dispersion of gaze within fixations. Lower values indicate more stable, tightly clustered fixations on stationary targets.

- **Target alignment:** Cosine similarity between gaze and target trajectories. Indicates how diligently participants followed targets with their eyes. Should be as close to 1 as possible.

We calculated the five indicators for all trials that followed (re)calibration of a given participant. For participant 21db28aa, we calculate it separately for trials 1-81 and 82-144 because they took a break after trial 81 but were not recalibrated, causing the data quality of the later trials to suffer. 

### Average and Maximum Validation Error

For each participant:
- Extract the average and maximum validation error from all validations inside the EyeLink ASCII file together with the corresponding trial numbers.

### Missing Data

For each participant:
- **Remove samples** before target onset and after target offset.
- **Calculate missing percentage** by counting the number of missing samples, dividing by the total number of samples, and multiplying by 100: 
   $$ \frac {n_{m}}{N} \cdot 100 $$
   Where ($n_{m}$) is the number of missing samples and ($N$) is the total number of samples.


### Precision

For each participant:

1. **Identify jumping dot trials** (trials containing fixations).

2. **Establish ground truth** using the algorithm described in the paper.

3. **Group fixation samples** according to interspersed saccades and blinks.

4. For each fixation group ($i$), **calculate the Root Mean Squared Error (RMSE)**:
   $$ \text{RMSE}_i = \sqrt{\frac{1}{N} \sum_{j=1}^{N} (x_j^2 + y_j^2)} $$
   where ($x_j$) and ($y_j$) are the gaze coordinates for the fixation group ($i$), and ($N$) is the number of samples in that group.

5. **Calculate the mean RMSE** across all fixation groups:
   $$ \text{Mean\ RMSE} = \frac{1}{M} \sum_{i=1}^{M} \text{RMSE}_i $$
   where ($M$) is the total number of fixation groups.
  

### Target Alignment

For each participant:
- **Preprocess gaze** to synchronize with target data but without additional steps like smoothing or blink removal.
- **Mask gaze and target data trajectories** by removing all samples with missing data.
- **Calculate cosine similarity** between the flattened target and gaze vectors:
   $$ \text{Cosine Similarity} = \frac{\text{gaze\_trajectory} \cdot \text{target\_trajectory}}{\|\text{gaze\_trajectory}\| \|\text{target\_trajectory}\|} $$

   where:
   - $\text{gaze\_trajectory} \cdot \text{target\_trajectory}$ is the dot product between the two vectors.
   - $\|\text{gaze\_trajectory}\| $ and $ \|\text{target\_trajectory}\| $ are the Euclidean norms of the gaze and target vectors, respectively.

In [2]:
# Prepare lists
ids = []
trials = []
val_error_avg = []
val_error_max = []
missing = []
precision = []
alignment = []

# Get all available participant IDs
participant_ids = get_participant_ids()

# Iterate over each participant ID
for participant_id in participant_ids:
    clear_output()

    # Load participant
    p = Participant(participant_id)

    # Extract EyeLink validation results
    validation_df = p.validation_check()
    validation_df = validation_df[~validation_df["first_trial"].isna() & ~validation_df["last_trial"].isna()]
    
    # For participant 21db28aa we need to run separately for trials 1-81 and 82-144
    # because they took a break after 81 and were not recalibrated causing their subsequent
    # trials to be messed up
    if p.id == "21db28aa":
        # Extract all metrics for trials 1-81
        ids.append(p.id)
        trials.append("1-81")
        missing.append(p.missing_data_check(trials=range(1, 82)))
        precision.append(p.fixation_precision_check(trials=range(1, 82)))
        alignment.append(p.target_alignment_check(trials=range(1, 82)))
        val_error_avg.append(validation_df["error_avg"].values[0])
        val_error_max.append(validation_df["error_max"].values[0])

        # Extract all metrics for trials 82-144 (without validation check)
        ids.append(p.id)
        trials.append("82-144")
        missing.append(p.missing_data_check(trials=range(82, 145)))
        precision.append(p.fixation_precision_check(trials=range(82, 145)))
        alignment.append(p.target_alignment_check(trials=range(82, 145)))
        val_error_avg.append("-")
        val_error_max.append("-")

    # For all other participants we extract all metrics following each 
    # separate calibration/ validation cycle
    else:
        # Iterate over each validation segment and select all corresponding trials
        for start, end, error_avg, error_max in zip(validation_df["first_trial"].astype(np.int16), validation_df["last_trial"].astype(np.int16), validation_df["error_avg"], validation_df["error_max"]):
            if start == 0:
                start = 1

            # Extract all metrics for the current validation segment
            ids.append(p.id)
            trials.append(f"{start}-{end}")
            missing.append(p.missing_data_check(trials=range(start, end + 1)))
            precision.append(p.fixation_precision_check(trials=range(start, end + 1)))
            alignment.append(p.target_alignment_check(trials=range(start, end + 1)))
            val_error_avg.append(error_avg)
            val_error_max.append(error_max)

# Put everything in a table
quality_table = pd.DataFrame({"Participant ID": ids,
                              "Trials": trials,
                              "AVG Validation Error (dva)": val_error_avg,
                              "MAX Validation Error (dva)": val_error_max,
                              "Missing Data (%)": np.round(missing, 2),
                              "Fixation Precision (dva)": np.round(precision, 2),
                              "Alignment (cosine similarity)": np.round(alignment, 2)})
quality_table = quality_table.sort_values(by=["Participant ID", "Trials"])

# checkout the table
clear_output()
quality_table.head(20)

,Participant ID,Trials,AVG Validation Error (dva),MAX Validation Error (dva),Missing Data (%),Fixation Precision (dva),Alignment (cosine similarity)
6,06b8d2d3,1-144,1.08,3.16,1.33,0.11,0.91
4,21db28aa,1-81,0.50,2.13,1.16,0.14,0.95
5,21db28aa,82-144,-,-,1.43,0.05,-0.21
2,68471e16,1-15,0.53,1.30,0.52,0.08,0.96
3,68471e16,16-144,0.80,1.22,0.48,0.09,0.91
1,6cde27b5,1-144,0.49,1.28,2.20,0.09,0.90
10,7d248f8f,1-144,0.30,0.84,0.12,0.13,0.94
0,88878fe6,1-144,0.45,1.64,2.29,0.09,0.91
11,cf910821,1-144,0.31,0.94,0.05,0.10,0.94
9,d2c0afa4,1-144,0.57,0.91,0.24,0.07,0.93
